In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image

def explain_and_visualize_histogram_equalization(output_excel_path="histogram_equalization_results.xlsx",
                                                 original_image_path="original_image.png",
                                                 equalized_image_path="equalized_image.png"):
    image = np.random.choice(range(30, 80), size=(10, 10))
    flat_image = image.flatten()
    unique_values = sorted(set(flat_image))
    histogram = {val: (flat_image == val).sum() for val in unique_values}
    total_pixels = len(flat_image)
    pdf = {val: freq / total_pixels for val, freq in histogram.items()}
    cdf = {}
    cumulative = 0
    for val in unique_values:
        cumulative += pdf[val]
        cdf[val] = cumulative
    L = 256
    new_values = {val: round((L - 1) * cdf[val]) for val in unique_values}
    transformed_image = np.vectorize(new_values.get)(image)
    transformed_histogram = {val: (transformed_image.flatten() == val).sum() for val in np.unique(transformed_image)}
    transformed_cdf = {}
    cumulative = 0
    for val in sorted(transformed_histogram.keys()):
        cumulative += transformed_histogram[val] / total_pixels
        transformed_cdf[val] = cumulative

    df = pd.DataFrame({
        "Intensity": unique_values,
        "Frequency": [histogram[val] for val in unique_values],
        "PDF": [pdf[val] for val in unique_values],
        "CDF": [cdf[val] for val in unique_values],
        "New Intensity": [new_values[val] for val in unique_values]
    })

    original_image = Image.fromarray(image.astype(np.uint8))
    equalized_image = Image.fromarray(transformed_image.astype(np.uint8))

    # Save images as PNG
    original_image.save(original_image_path)
    equalized_image.save(equalized_image_path)

    plt.figure(figsize=(16, 8))

    plt.subplot(2, 4, 1)
    plt.imshow(original_image, cmap='gray', vmin=0, vmax=255)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(2, 4, 2)
    plt.bar(histogram.keys(), histogram.values(), color='blue', alpha=0.7)
    plt.title("Original Histogram")
    plt.xlabel("Intensity")
    plt.ylabel("Frequency")

    plt.subplot(2, 4, 3)
    plt.plot(unique_values, [cdf[val] for val in unique_values], color='blue', marker='o')
    plt.title("Original CDF")
    plt.xlabel("Intensity")
    plt.ylabel("Cumulative Probability")

    plt.subplot(2, 4, 5)
    plt.imshow(equalized_image, cmap='gray', vmin=0, vmax=255)
    plt.title("Equalized Image")
    plt.axis('off')

    plt.subplot(2, 4, 6)
    plt.bar(transformed_histogram.keys(), transformed_histogram.values(), color='green', alpha=0.7)
    plt.title("Equalized Histogram")
    plt.xlabel("Intensity")
    plt.ylabel("Frequency")

    plt.subplot(2, 4, 7)
    plt.plot(list(transformed_cdf.keys()), list(transformed_cdf.values()), color='purple', marker='o')
    plt.title("Equalized CDF")
    plt.xlabel("Intensity")
    plt.ylabel("Cumulative Probability")

    plt.tight_layout()
    plt.show()

    # Save DataFrame to Excel
    with pd.ExcelWriter(output_excel_path) as writer:
        df.to_excel(writer, index=False, sheet_name="Histogram Equalization")

explain_and_visualize_histogram_equalization("histogram_equalization_results.xlsx",
                                             "original_image.png",
                                             "equalized_image.png")
